# NMR Inversion Recovery homework

<strong>Author(s):</strong> Seth D. Veenbaas, Jessica A. Nash, The Molecular Sciences Software Institute

<div class="alert alert-block alert-info"> 
<h2>Overview</h2>

<strong>Objective:</strong>

* Use Pandas and Scipy to proceed NMR data.

* Calculate the $T_1$ relaxation times.

* Determine if solvent conditions affect $T_1$ relaxation times.

</div>

## Inversion recovery experiment
The inversion-recovery experiment measures $T_1$ relaxation times of any nucleus. If the net magnetization is placed along the -z axis, it will gradually return to its equilibrium position along the +z axis at a rate governed by $T_1$. The equation governing this behavior as a function of the time t after its displacement is: 

$$ M_z(t) = M_{z,\text{eq}} \cdot \left(1 - 2e^{-t/T_1}\right) $$

The basic pulse sequence consists of an 180<sup>°</sup> pulse that inverts the magnetization to the -z axis. During the following delay, relaxation along the longitudinal plane takes place. Magnetization comes back to the original equilibrium z-magnetization. A 90<sup>°</sup> pulse creates transverse magnetization. The experiment is repeated for a series of delay values taken from a variable delay list. A 1D spectrum is obtained for each value of vd and stored in a pseudo 2D dataset. The longer the relaxation delay (d<sub>1</sub>) is, the more precise the T<sub>1</sub> measurement is. An ideal relaxation time (d<sub>1</sub>) can be calculated (aq = acquisition time):

 $$ d_1 + \text{aq} = 5 \cdot T_1 $$

 <br>

![image.png](../images/t1_relaxation_pulse_sequence.png)

More information:
https://imserc.northwestern.edu/downloads/nmr-t1.pdf



## Importing Required Libraries

First, let's import the python libraries/packages we need to work with the data.

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import mnova

Let's load, reformat, and view our data for Ibuprofen inversion recovery.

In [7]:
# Load the data from the CSV file
ibuprofen_inversion_data = pd.read_csv('../data/Ibuprofen-C13-invrec-data-mnova.csv', header=1)

# Runs reformating function
ibuprofen_inversion_data = mnova.rename_columns(ibuprofen_inversion_data)

# Display the first 5 row of the dataframe
ibuprofen_inversion_data.head()

,Time(s),183.7_ppm,143.4_ppm,139.3_ppm,132.3_ppm,130.3_ppm,47.5_ppm,24.9_ppm
1,0.0625,-4684.674918,-5499.794916,-5036.192817,-6258.796206,-5560.295376,-1331.859019,-2792.253255
2,0.1250,-3658.108805,-6311.520261,-5046.870771,-5070.043928,-5501.243266,-2232.157177,-718.401400
3,0.2500,-3931.386081,-4594.110752,-6585.526371,-4965.122729,-4204.977913,-4227.647119,-2714.233721
4,0.5000,-3852.769938,-3103.076230,-4563.784884,-3098.568237,-3719.213980,-2123.348926,-399.047348
5,1.0000,-4292.297168,-3898.901847,-6335.422168,-3270.550247,-2361.015131,600.838812,234.863806


<div class="alert alert-block alert-warning">
<h3>Step 1</h3>

Load, reformat, and view the inversion recovery data for ibuprofen in **Condition X** and **Condition Y**:

- **Condition X**: `'../data/Ibuprofen-Condition-X-mnova.csv'`
- **Condition Y**: `'../data/Ibuprofen-Condition-Y-data-mnova.csv'`

</div>

In [ ]:
# Load the data from the Condition X CSV file
ibuprofen_inversion_data = pd.read_csv('../data/Ibuprofen-Condition-X-mnova.csv', header=1)

# Runs reformating function
ibuprofen_inversion_data = mnova.rename_columns(ibuprofen_inversion_data)

# Display the first 5 row of the dataframe
ibuprofen_inversion_data.head()

In [ ]:
# Load the data from the Condition Y CSV file
ibuprofen_inversion_data = pd.read_csv('../data/Ibuprofen-Condition-Y-data-mnova.csv', header=1)

# Runs reformating function
ibuprofen_inversion_data = mnova.rename_columns(ibuprofen_inversion_data)

# Display the first 5 row of the dataframe
ibuprofen_inversion_data.head()

Let's define the functions you'll need to calculate $T_1$!

In [ ]:
# Define the inversion recovery model with the parameters (M, T1, and C)
def inversion_recovery_model(time, M, T1, C):
    return M * (1 - 2 * np.exp(-time / T1)) + C

# Define function to fit data to a inversion recovery model
def fit_relaxation_data(time_data, peak_data):
    # Initial guess for M, T1, and C
    initial_guess = [max(peak_data), 1.0, min(peak_data)]
    
    # Fit the curve
    param_optimal, _ = curve_fit(inversion_recovery_model, time_data, peak_data, p0=initial_guess)
    
    return param_optimal

<div class="alert alert-block alert-warning">
<h3>Step 2</h3>

We need to use following `for` loop for each of our data sets. Modify the code in the following cell to be a reusable function. Incorporate the follow:

#### 1. Function Name: `calc_t1`

#### 2. Parameters:

- `df` (DataFrame): Inversion recovery data.

#### 3. Function body (indented code):

```python
# Create an empty DataFrame for t1_data with columns 'Peak' and 'T1(s)'
t1_data = pd.DataFrame(columns=['Peak', 'T1(s)'])

# Calculate T1 for each peak and plot the fit
for column in df.columns[1:]:
    
    # Fit relaxation data to calculate T1
    time_data = df['Time(s)']
    peak_data = df[column]
    param_optimal = fit_relaxation_data(time_data, peak_data)
    T1 = param_optimal[1]
    
    # Add T1 time to the t1_data DataFrame
    new_row = pd.DataFrame({'Peak': [column], 'T1(s)': [T1]})
    t1_data = pd.concat([t1_data, new_row], ignore_index=True)
```

#### 4. Return:

- `t1_data` (DataFrame): T1 relaxation times for every NMR peak.

</div>

### Here is what it looks like to define a function:
```python
def function_name(parameters):
    # Optional: explain what your function does in a Docstring
    """
    Docstring
    """
    # Function body (indented code)
    return output  # Optional: Return a result
```

In [ ]:
# Calculate T1 times from data in a Dataframe    
def calc_t1(df):    
    # Create an empty DataFrame for t1_data with columns 'Peak' and 'T1(s)'
    t1_data = pd.DataFrame(columns=['Peak', 'T1(s)'])
    
    # Calculate T1 for each peak and plot the fit
    for column in df.columns[1:]:
        
        # Fit relaxation data to calculate T1
        time_data = df['Time(s)']
        peak_data = df[column]
        param_optimal = fit_relaxation_data(time_data, peak_data)
        T1 = param_optimal[1]
        
        # Add T1 time to the t1_data DataFrame
        new_row = pd.DataFrame({'Peak': [column], 'T1(s)': [T1]})
        t1_data = pd.concat([t1_data, new_row], ignore_index=True)
    
    return t1_data
    

<div class="alert alert-block alert-warning">
<h3>Step 3</h3>

Use your `calc_t1` function to calculate $T_1$ times for ibuprofen in each of the solvent conditions and save the results to their own variables.

Display the results.

</div>

In [ ]:
ibuprofen_conidtion1_t1_data = calc_t1(ibuprofen_conidtion1)
ibuprofen_conidtion2_t1_data = calc_t1(ibuprofen_conidtion2)
ibuprofen_conidtion3_t1_data = calc_t1(ibuprofen_conidtion3)

In [ ]:
ibuprofen_conidtion1_t1_data

In [ ]:
ibuprofen_conidtion2_t1_data

In [ ]:
ibuprofen_conidtion3_t1_data

<div class="alert alert-block alert-warning">
<h3>Step 4</h3>

Which condition increased $T_1$ relaxation times the most?

What was the highest $T_1$ time for peak 143.4_ppm?

</div>

In [ ]:
condition=
highest_T1_peak_143_ppm=